## Import

In [ ]:
# 한글 출력을 위한 나눔글꼴 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
# catboost 설치
!pip install catboost

In [ ]:
# Optuna 설치
!pip install optuna

In [ ]:
import optuna

import pandas as pd
import numpy as np

# 경고창 무시
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss

from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

from sklearn import set_config
set_config(transform_output="pandas")

# 재현성을 위한 random seed
import numpy as np
import random
import os

def my_seed_everywhere(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # numpy
    os.environ["PYTHONHASHSEED"] = str(seed) # os

my_seed = 42
my_seed_everywhere(my_seed)

import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

## Data Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/data'

train = pd.read_csv(path+'/train.csv').drop(columns=['ID'])
test = pd.read_csv(path+'/test.csv').drop(columns=['ID'])

In [ ]:
train

In [ ]:
train.info()

## EDA

### 타깃 비율 확인

In [ ]:
train['임신 성공 여부'].mean()

즉, 약 25.8%가 임신에 성공했다.

### 전반적인 결측치 확인

In [ ]:
train.isnull().sum().sort_values(ascending=False)[:33]

- 특정 feature들에서 동일한 개수의 결측치(6291개)가 발견이 되므로 이 부분을 따로 확인한다.
- 6291개 이상의 행에서 결측치를 가지는 열은 30개이며, 그 중 21개 열은 결측치가 있는 행이 6291개이다.

In [ ]:
train_6291 = train[train['미세주입에서 생성된 배아 수'].isnull()]
train_6291

In [ ]:
train_6291.isnull().sum().sort_values(ascending=False)[:33]

- 6291개 이상의 결측치를 가지는 30개의 열 중 '임신 시도 또는 마지막 임신 경과 연수'를 제외한 나머지는 모두 공통적으로 결측치가 된 것이다.

In [ ]:
train_6291['시술 유형'].value_counts()

In [ ]:
train[train['미세주입에서 생성된 배아 수'].notnull()]['시술 유형'].value_counts()

즉, 열의 모든 행에 결측치가 있는 것은 시술 유형이 모두 DI이고 결측치가 없는 것은 IVF이다.
- 시술 유형이 DI이면 상당수의 열이 결측치가 되고, IVF이면 값이 존재하므로, 이 자체가 중요한 특성이 된다.

In [ ]:
train_6291['정자 출처'].value_counts()

In [ ]:
train_6291['난자 출처'].value_counts()

결론적으로 DI 데이터는
- 정자 출처가 기증 제공
- 난자 출처가 알 수 없음
- 시험관 방식과 연관이 없다. (인공 수정)

즉, 정자 기증 방식의 인공 수정 방식이라서 많은 열에서 100% 행에 결측치가 있다.

In [ ]:
train_6291['특정 시술 유형'].value_counts()

In [ ]:
col_DI_missing = train_6291.columns[train_6291.isnull().sum() == 6291]
col_DI_missing

### 모두 같은 값을 가지는 열 확인

In [ ]:
train.loc[:, train.nunique()==1]

In [ ]:
train.loc[:, train.nunique()==1].describe()

- 착상 전 유전 검사 사용 여부: 2718개 행은 모두 1이고 나머지는 모두 결측치이다.
- 불임 원인 여성 요인: 모든 행이 0이다. (제거)
- PGD 시술 여부: 2179개 행은 모두 1이고 나머지는 결측치이다.
- PGS 시술 여부: 1929개 행은 모두 1이고 나머지는 결측치이다.
- 난자 채취 경과일: 198863개의 행은 모두 0이고 나머지는 결측치이다.

In [ ]:
train.drop(columns=['불임 원인 - 여성 요인'], inplace=True)
test.drop(columns=['불임 원인 - 여성 요인'], inplace=True)

In [ ]:
PGS_PGD = train.loc[:, ['착상 전 유전 검사 사용 여부', 'PGS 시술 여부', '착상 전 유전 진단 사용 여부', 'PGD 시술 여부']]

In [ ]:
PGS_PGD.describe()

In [ ]:
PGS_PGD['착상 전 유전 진단 사용 여부'].sum()

In [ ]:
PGS_PGD.loc[(PGS_PGD['착상 전 유전 검사 사용 여부'] == 1) & (PGS_PGD['PGS 시술 여부'] == 1)]

In [ ]:
PGS_PGD.loc[(PGS_PGD['착상 전 유전 진단 사용 여부'] == 1) & (PGS_PGD['PGD 시술 여부'] == 1)]

- 즉, PGS가 시행되었지만 PGS 치료로 분류되지는 않는 행과 PGD가 시행되었지만 PGD 치료로 분류되지는 행이 모두 존재한다.
- 해당 치료로 분류되는지 여부보다는 실제 시행 여부가 중요하다고 판단했기에 두 열은 제거한다.

In [ ]:
train.drop(columns=['PGS 시술 여부', 'PGD 시술 여부'], inplace=True)
test.drop(columns=['PGS 시술 여부', 'PGD 시술 여부'], inplace=True)

### 그래프 시각화

In [ ]:
train

#### 시술 시기, 나이

In [ ]:
sns.barplot(x='시술 시기 코드', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='시술 당시 나이', y='임신 성공 여부', data=train,
            order=['만18-34세', '만35-37세', '만38-39세', '만40-42세', '만43-44세', '알 수 없음'])
plt.show()

- 시술자의 나이가 많을수록 임신 성공률이 떨어지는 유의미한 경향성을 보인다.

#### 시술 유형, 경과 연수, 세부 유형

In [ ]:
sns.barplot(x='시술 유형', y='임신 성공 여부', data=train)
plt.show()

- 두 시술 유형 사이의 성공률은 유의미한 차이가 난다.

In [ ]:
sns.barplot(x='임신 시도 또는 마지막 임신 경과 연수', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='특정 시술 유형', y='임신 성공 여부', data=train)
plt.show()

- 실제로 차이가 난다는 것은 확인 가능하지만, 복합/조합으로 된 것이 있으므로 범주가 많이 나온다.

#### 배란 자극, 유도 유형

In [ ]:
sns.barplot(x='배란 자극 여부', y='임신 성공 여부', data=train)
plt.show()

- 배란 자극을 하는 쪽이 성공률이 더 높았다.

In [ ]:
sns.barplot(x='배란 유도 유형', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
train[train['배란 자극 여부'] == 0]['배란 유도 유형'].value_counts()

In [ ]:
train[train['배란 자극 여부'] == 1]['배란 유도 유형'].value_counts()

- 이 부분은 기록되지 않은 시행, 알 수 없음 행이 너무 많아 유의미한 정보 파악이 어렵다. 그래서 이 열은 제거하도록 한다.

In [ ]:
train.drop(columns=['배란 유도 유형'], inplace=True)
test.drop(columns=['배란 유도 유형'], inplace=True)

#### 단일 배아 이식 (+추가 확인)

In [ ]:
sns.barplot(x='단일 배아 이식 여부', y='임신 성공 여부', data=train)
plt.show()

- DI 유형에서는 이 값이 결측치이므로 모두 IVF 유형에서의 값들이다.
- 단일 배아 이식했을 때 성공률이 유의미하게 높았는데, 통상 관념과는 다르므로 확인 필요

In [ ]:
train[train['단일 배아 이식 여부']==0]['시술 당시 나이'].value_counts(normalize=True)

In [ ]:
train[train['단일 배아 이식 여부']==1]['시술 당시 나이'].value_counts(normalize=True)

- 단일 배아 이식을 하지 않은 경우, 만 18-34세가 약 35%, 만 35-37세가 약 22%여서 전체의 약 57% 정도가 임신 성공률이 평균 이상에 해당하는 37세 미만인 것으로 보인다.
- 단일 배아 이식을 한 경우, 만 18-34세가 약 56%, 만 35-37세가 약 24%여서 전체의 약 80% 정도가 임신 성공률이 평균 이상에 해당하는 37세 미만인 것으로 보인다.
- 시술 당시 나이의 비율이 차이가 나는 것으로 보아 이 부분과 연관이 있을 것으로 생각된다.

#### 유전 검사, 유전 진단 (+추가 확인)

In [ ]:
sns.barplot(x='착상 전 유전 검사 사용 여부', y='임신 성공 여부', data=train)
plt.show()

- DI 유형에서는 이 값이 결측치이므로 모두 IVF 유형에서의 값들이다. (IVF에서도 결측치 다수 존재)
- 전체 평균 성공률, IVF 평균 성공률보다 성공률이 유의미하게 낮았는데, 통상 관념과는 다르므로 확인 필요

In [ ]:
train[train['착상 전 유전 검사 사용 여부']==1]['시술 당시 나이'].value_counts(normalize=True)

- 만 18-34세가 약 15%, 만 35-37세가 약 17%여서 전체의 약 1/3 정도가 임신 성공률이 평균 이상에 해당하는 37세 미만인 것이 원인으로 보인다.

In [ ]:
sns.barplot(x='착상 전 유전 진단 사용 여부', y='임신 성공 여부', data=train)
plt.show()

- 유전 진단을 사용한 쪽의 성공률이 유의미하게 낮았는데, 통상 관념과는 다르므로 확인 필요

In [ ]:
train[train['착상 전 유전 진단 사용 여부']==1]['시술 당시 나이'].value_counts(normalize=True)

- 만 18-34세가 약 55%, 만 35-37세가 약 26%여서 전체의 약 81% 정도가 임신 성공률이 평균 이상에 해당하는 37세 미만이다.
- 나이가 아닌 다른 외부 요인이 영향을 끼쳤을 것으로 보인다.

In [ ]:
train[train['착상 전 유전 진단 사용 여부']==1]['총 생성 배아 수'].value_counts(normalize=True)

- 적정 생성 배아 수보다 적은 비율이 높은 것으로 보인다.

#### 불임 원인

In [ ]:
sns.barplot(x='남성 주 불임 원인', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='남성 부 불임 원인', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='여성 주 불임 원인', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='여성 부 불임 원인', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='부부 주 불임 원인', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='부부 부 불임 원인', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='불명확 불임 원인', y='임신 성공 여부', data=train)
plt.show()

- 불명확 불임 원인은 큰 차이를 파악하기 어렵다.
- 나머지는 불임 원인이 있으면 유의미하게 임신 성공률이 낮았다.

#### 세부 불임 원인

In [ ]:
sns.barplot(x='불임 원인 - 난관 질환', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='불임 원인 - 남성 요인', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='불임 원인 - 배란 장애', y='임신 성공 여부', data=train)
plt.show()

- 여성 요인은 단일값이어서 사전에 제거하였다.

In [ ]:
sns.barplot(x='불임 원인 - 자궁경부 문제', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
train['불임 원인 - 자궁경부 문제'].sum()

In [ ]:
sns.barplot(x='불임 원인 - 자궁내막증', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='불임 원인 - 정자 농도', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='불임 원인 - 정자 면역학적 요인', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
train['불임 원인 - 정자 면역학적 요인'].sum()

In [ ]:
sns.barplot(x='불임 원인 - 정자 운동성', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='불임 원인 - 정자 형태', y='임신 성공 여부', data=train)
plt.show()

- 전반적으로 유의미한 차이를 보이지 않거나 통상 관념과 다른 결과를 보이거나, 그렇지 않더라도 신뢰구간이 매우 긴 모습을 보였다.
- 이는 1에 해당하는 도수가 매우 적은 것이 원인일 것이다.

#### 배아 생성 주요 이유

In [ ]:
sns.barplot(x='배아 생성 주요 이유', y='임신 성공 여부', data=train)
plt.show()

#### 시술 횟수

In [ ]:
sns.barplot(x='총 시술 횟수', y='임신 성공 여부', data=train,
            order=['0회', '1회', '2회', '3회', '4회', '5회', '6회 이상'])
plt.show()

In [ ]:
sns.barplot(x='클리닉 내 총 시술 횟수', y='임신 성공 여부', data=train,
            order=['0회', '1회', '2회', '3회', '4회', '5회', '6회 이상'])
plt.show()

In [ ]:
sns.barplot(x='IVF 시술 횟수', y='임신 성공 여부', data=train,
            order=['0회', '1회', '2회', '3회', '4회', '5회', '6회 이상'])
plt.show()

In [ ]:
sns.barplot(x='DI 시술 횟수', y='임신 성공 여부', data=train,
            order=['0회', '1회', '2회', '3회', '4회', '5회', '6회 이상'])
plt.show()

- DI를 제외한 나머지에서는 이미 수행했던 시술 횟수가 많을수록 임신 성공률이 떨어지는 경향성을 보였다.

#### 임신 횟수

In [ ]:
sns.barplot(x='총 임신 횟수', y='임신 성공 여부', data=train,
            order=['0회', '1회', '2회', '3회', '4회', '5회', '6회 이상'])
plt.show()

In [ ]:
sns.barplot(x='IVF 임신 횟수', y='임신 성공 여부', data=train,
            order=['0회', '1회', '2회', '3회', '4회', '5회', '6회 이상'])
plt.show()

In [ ]:
sns.barplot(x='DI 임신 횟수', y='임신 성공 여부', data=train,
            order=['0회', '1회', '2회', '3회', '4회', '5회', '6회 이상'])
plt.show()

- 임신 횟수는 도수가 적어서 값이 튀는 것 이외에는 유의미한 차이를 발견하기 어렵다.

#### 출산 횟수

In [ ]:
sns.barplot(x='총 출산 횟수', y='임신 성공 여부', data=train,
            order=['0회', '1회', '2회', '3회', '4회', '5회', '6회 이상'])
plt.show()

In [ ]:
sns.barplot(x='IVF 출산 횟수', y='임신 성공 여부', data=train,
            order=['0회', '1회', '2회', '3회', '4회', '5회', '6회 이상'])
plt.show()

In [ ]:
sns.barplot(x='DI 출산 횟수', y='임신 성공 여부', data=train,
            order=['0회', '1회', '2회', '3회', '4회', '5회', '6회 이상'])
plt.show()

- 출산 횟수는 도수가 적어서 값이 튀는 것 이외에는 유의미한 차이를 발견하기 어렵다.

#### 배아 수, 난자 수

In [ ]:
sns.barplot(x='총 생성 배아 수', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='미세주입된 난자 수', y='임신 성공 여부', data=train)
plt.show()

- DI 유형에서는 이 값이 결측치이므로 모두 IVF 유형에서의 값들이다.
- 10~15개 정도일 때 성공률이 제일 높은 것으로 보인다.

In [ ]:
sns.barplot(x='미세주입에서 생성된 배아 수', y='임신 성공 여부', data=train)
plt.show()

- DI 유형에서는 이 값이 결측치이므로 모두 IVF 유형에서의 값들이다.
- 특정 범위에서 성공률이 제일 높은 것으로 보인다. 그 범위를 벗어나면 점점 떨어지는 것으로 확인되었다.

In [ ]:
sns.barplot(x='이식된 배아 수', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='미세주입 배아 이식 수', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='저장된 배아 수', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='미세주입 후 저장된 배아 수', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='해동된 배아 수', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='해동 난자 수', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='수집된 신선 난자 수', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='혼합된 난자 수', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='파트너 정자와 혼합된 난자 수', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='기증자 정자와 혼합된 난자 수', y='임신 성공 여부', data=train)
plt.show()

- DI 유형에서는 이 값이 결측치이므로 모두 IVF 유형에서의 값들이다.
- 특정 범위에서 성공률이 제일 높은 것으로 보인다. 그 범위를 벗어나면 점점 떨어지는 것으로 확인되었다.

#### 난자, 정자 출처/기증자 나이

In [ ]:
sns.barplot(x='난자 출처', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
train['난자 출처'].value_counts()

- 난자 출처 알 수 없음 = DI 유형

In [ ]:
sns.barplot(x='정자 출처', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
train['정자 출처'].value_counts()

In [ ]:
sns.barplot(x='난자 기증자 나이', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
train['난자 기증자 나이'].value_counts()

In [ ]:
sns.barplot(x='정자 기증자 나이', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
train['정자 기증자 나이'].value_counts()

- 기증자 나이는 유의미한 차이를 불러오지는 못하지만 만 20세 이하는 약간 낮은 것으로 보인다.

#### 동결, 신선, 기증 배아 사용 여부

In [ ]:
sns.barplot(x='동결 배아 사용 여부', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='신선 배아 사용 여부', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='기증 배아 사용 여부', y='임신 성공 여부', data=train)
plt.show()

- DI 유형에서는 이 값이 결측치이므로 모두 IVF 유형에서의 값들이다.
- 동결 배아를 사용한 쪽이 약간 낮았다.
- 신선 배아를 사용한 쪽이 약간 높았다.
- 기증 배아를 사용한 쪽이 약간 높았다.

#### 대리모 여부

In [ ]:
sns.barplot(x='대리모 여부', y='임신 성공 여부', data=train)
plt.show()

- 대리모 여부는 큰 차이를 보이지는 않는다.

#### 난자 해동, 혼합 / 배아 이식, 해동

In [ ]:
sns.barplot(x='난자 해동 경과일', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
train['난자 해동 경과일'].value_counts()

In [ ]:
sns.barplot(x='난자 혼합 경과일', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='배아 이식 경과일', y='임신 성공 여부', data=train)
plt.show()

In [ ]:
sns.barplot(x='배아 해동 경과일', y='임신 성공 여부', data=train)
plt.show()

## Data Pre-processing

### 세부 결측치 확인

In [ ]:
train.info()

In [ ]:
col_DI_missing

- col_DI_missing은 적어도 시술 유형이 DI인 행이 모두 결측치인 경우이다.
- 여기서 이미 제외한 '난자 채취 경과일', 'PGS 시술 여부', 'PGD 시술 여부'을 제외한 26개의 결측치를 유심히 볼 것이다.

['단일 배아 이식 여부', '착상 전 유전 진단 사용 여부', '배아 생성 주요 이유', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수',
       '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수',
       '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부',
       '대리모 여부']

- 특히 이 21개의 열은 모두 동시에 결측치가 됨과 동시에 결측치가 아닌 부분도 모두 동시이므로 이 부분은 별도로 결측치 처리를 하지 않는다. (CatBoost 사용 예정이므로 별도로 처리하지 않을 예정)

In [ ]:
col_missing = train.columns[train.isnull().sum() / len(train) > 0]
col_missing

결측치를 다루어야 하는 열은 총 28개이다

이 중에서 이미 결측치를 처리하지 않기로 한 21개 열을 제외한
- 임신 시도 또는 마지막 임신 경과 연수
- 특정 시술 유형
- 착상 전 유전 검사 사용 여부
- 난자 해동 경과일
- 배아 해동 경과일
- 난자 혼합 경과일
- 배아 이식 경과일

중에서 결측치를 채우면 데이터 왜곡이 일어날 수 있는 열들을 제외한
- 특정 시술 유형
- 착상 전 유전 검사 사용 여부

는 결측치를 채우도록 한다.

In [ ]:
def fill_missing(train, test):
    ## 결측치는 특정 시술 유형을 알 수 없으므로 Unknown으로 채운다.
    train['특정 시술 유형'].fillna('Unknown', inplace=True)
    test['특정 시술 유형'].fillna('Unknown', inplace=True)

    ## 이 부분은 사용 여부가 0으로 추정되므로 0으로 채운다.
    train['착상 전 유전 검사 사용 여부'].fillna(0, inplace=True)
    test['착상 전 유전 검사 사용 여부'].fillna(0, inplace=True)

In [ ]:
fill_missing(train, test)

In [ ]:
train.info()

### 데이터 정리

In [ ]:
train.info()

In [ ]:
train['특정 시술 유형'].value_counts()

In [ ]:
train['배아 생성 주요 이유'].value_counts()

In [ ]:
train['정자 출처'].value_counts()

In [ ]:
train['난자 출처'].value_counts()

In [ ]:
cat_features = ['시술 시기 코드', '시술 당시 나이_불명', '임신 시도 또는 마지막 임신 경과 연수_결측', '시술 유형', # '특정 시술 유형',
                '특정_IVF', '특정_ICSI', '특정_IUI', '특정_ICI', '특정_GIFT', '특정_FER', '특정_Generic DI', '특정_IVI', '특정_BLASTOCYST', '특정_AH', '특정_Unknown',
                '단일 배아 이식 여부', '단일 배아 이식 여부_결측', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '착상 전 유전 진단 사용 여부_결측',
                '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인',
                '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증',
                '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태',
                '배아생성_기증용', '배아생성_난자저장용', '배아생성_배아저장용', '배아생성_현재시술용', '배아생성_연구용',
                '총 생성 배아 수_결측', '미세주입된 난자 수_결측', '미세주입에서 생성된 배아 수_결측', '이식된 배아 수_결측', '미세주입 배아 이식 수_결측',
                '저장된 배아 수_결측', '미세주입 후 저장된 배아 수_결측', '해동된 배아 수_결측', '해동 난자 수_결측', '수집된 신선 난자 수_결측', '저장된 신선 난자 수_결측',
                '혼합된 난자 수_결측', '파트너 정자와 혼합된 난자 수_결측', '기증자 정자와 혼합된 난자 수_결측',
                '난자 출처', '정자 출처', '난자 기증자 나이_불명', '정자 기증자 나이_불명',
                '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부',
                '동결 배아 사용 여부_결측', '신선 배아 사용 여부_결측', '기증 배아 사용 여부_결측', '대리모 여부_결측',
                '난자 해동 경과일_결측', '난자 혼합 경과일_결측', '배아 이식 경과일_결측', '배아 해동 경과일_결측', '난자 채취 경과일_결측']

In [ ]:
def cleansing(df):
    age_dict = {'알 수 없음': -1, '만18-34세': 0, '만35-37세': 1, '만38-39세': 2,
                '만40-42세': 3, '만43-44세': 4, '만45-50세': 5}

    df['시술 당시 나이'] = [age_dict[age] for age in df['시술 당시 나이']]
    df['시술 당시 나이_불명'] = df['시술 당시 나이'].apply(lambda x: 1 if x == -1 else 0)

    df['임신 시도 또는 마지막 임신 경과 연수'] = df['임신 시도 또는 마지막 임신 경과 연수'].fillna(-1)
    df['임신 시도 또는 마지막 임신 경과 연수_결측'] = df['임신 시도 또는 마지막 임신 경과 연수'].apply(lambda x: 1 if x == -1 else 0)

    df['단일 배아 이식 여부'] = df['단일 배아 이식 여부'].fillna(-1)
    df['단일 배아 이식 여부_결측'] = df['단일 배아 이식 여부'].apply(lambda x: 1 if x == -1 else 0)

    df['착상 전 유전 진단 사용 여부'] = df['착상 전 유전 진단 사용 여부'].fillna(-1)
    df['착상 전 유전 진단 사용 여부_결측'] = df['착상 전 유전 진단 사용 여부'].apply(lambda x: 1 if x == -1 else 0)

    df['특정_IVF'] = [1 if 'IVF' in tp else 0 for tp in df['특정 시술 유형']]
    df['특정_ICSI'] = [1 if 'ICSI' in tp else 0 for tp in df['특정 시술 유형']]
    df['특정_IUI'] = [1 if 'IUI' in tp else 0 for tp in df['특정 시술 유형']]
    df['특정_ICI'] = [1 if 'ICI' in tp else 0 for tp in df['특정 시술 유형']]
    df['특정_GIFT'] = [1 if 'GIFT' in tp else 0 for tp in df['특정 시술 유형']]
    df['특정_FER'] = [1 if 'FER' in tp else 0 for tp in df['특정 시술 유형']]
    df['특정_Generic DI'] = [1 if 'Generic DI' in tp else 0 for tp in df['특정 시술 유형']]
    df['특정_IVI'] = [1 if 'IVI' in tp else 0 for tp in df['특정 시술 유형']]
    df['특정_BLASTOCYST'] = [1 if 'BLASTOCYST' in tp else 0 for tp in df['특정 시술 유형']]
    df['특정_AH'] = [1 if 'AH' in tp else 0 for tp in df['특정 시술 유형']]
    df['특정_Unknown'] = [1 if 'Unknown' in tp else 0 for tp in df['특정 시술 유형']]

    df['배아생성_기증용'] = [1 if isinstance(reason, str) and reason == '기증용' in reason else 0 for reason in df['배아 생성 주요 이유']]
    df['배아생성_난자저장용'] = [1 if isinstance(reason, str) and reason == '난자 저장용' in reason else 0 for reason in df['배아 생성 주요 이유']]
    df['배아생성_배아저장용'] = [1 if isinstance(reason, str) and reason == '배아 저장용' in reason else 0 for reason in df['배아 생성 주요 이유']]
    df['배아생성_연구용'] = [1 if isinstance(reason, str) and reason == '연구용' in reason else 0 for reason in df['배아 생성 주요 이유']]
    df['배아생성_현재시술용'] = [1 if isinstance(reason, str) and reason == '현재 시술용' in reason else 0 for reason in df['배아 생성 주요 이유']]

    count_dict = {'0회': 0, '1회': 1, '2회': 2, '3회': 3,
                  '4회': 4, '5회': 5, '6회 이상': 6}

    df['총 시술 횟수'] = [count_dict[count] for count in df['총 시술 횟수']]
    df['클리닉 내 총 시술 횟수'] = [count_dict[count] for count in df['클리닉 내 총 시술 횟수']]
    df['IVF 시술 횟수'] = [count_dict[count] for count in df['IVF 시술 횟수']]
    df['DI 시술 횟수'] = [count_dict[count] for count in df['DI 시술 횟수']]
    df['총 임신 횟수'] = [count_dict[count] for count in df['총 임신 횟수']]
    df['IVF 임신 횟수'] = [count_dict[count] for count in df['IVF 임신 횟수']]
    df['DI 임신 횟수'] = [count_dict[count] for count in df['DI 임신 횟수']]
    df['총 출산 횟수'] = [count_dict[count] for count in df['총 출산 횟수']]
    df['IVF 출산 횟수'] = [count_dict[count] for count in df['IVF 출산 횟수']]
    df['DI 출산 횟수'] = [count_dict[count] for count in df['DI 출산 횟수']]

    df['총 생성 배아 수'] = df['총 생성 배아 수'].fillna(-1)
    df['미세주입된 난자 수'] = df['미세주입된 난자 수'].fillna(-1)
    df['미세주입에서 생성된 배아 수'] = df['미세주입에서 생성된 배아 수'].fillna(-1)
    df['이식된 배아 수'] = df['이식된 배아 수'].fillna(-1)
    df['미세주입 배아 이식 수'] = df['미세주입 배아 이식 수'].fillna(-1)
    df['저장된 배아 수'] = df['저장된 배아 수'].fillna(-1)
    df['미세주입 후 저장된 배아 수'] = df['미세주입 후 저장된 배아 수'].fillna(-1)
    df['해동된 배아 수'] = df['해동된 배아 수'].fillna(-1)
    df['해동 난자 수'] = df['해동 난자 수'].fillna(-1)
    df['수집된 신선 난자 수'] = df['수집된 신선 난자 수'].fillna(-1)
    df['저장된 신선 난자 수'] = df['저장된 신선 난자 수'].fillna(-1)
    df['혼합된 난자 수'] = df['혼합된 난자 수'].fillna(-1)
    df['파트너 정자와 혼합된 난자 수'] = df['파트너 정자와 혼합된 난자 수'].fillna(-1)
    df['기증자 정자와 혼합된 난자 수'] = df['기증자 정자와 혼합된 난자 수'].fillna(-1)

    df['총 생성 배아 수_결측'] = df['총 생성 배아 수'].apply(lambda x: 1 if x == -1 else 0)
    df['미세주입된 난자 수_결측'] = df['미세주입된 난자 수'].apply(lambda x: 1 if x == -1 else 0)
    df['미세주입에서 생성된 배아 수_결측'] = df['미세주입에서 생성된 배아 수'].apply(lambda x: 1 if x == -1 else 0)
    df['이식된 배아 수_결측'] = df['이식된 배아 수'].apply(lambda x: 1 if x == -1 else 0)
    df['미세주입 배아 이식 수_결측'] = df['미세주입 배아 이식 수'].apply(lambda x: 1 if x == -1 else 0)
    df['저장된 배아 수_결측'] = df['저장된 배아 수'].apply(lambda x: 1 if x == -1 else 0)
    df['미세주입 후 저장된 배아 수_결측'] = df['미세주입 후 저장된 배아 수'].apply(lambda x: 1 if x == -1 else 0)
    df['해동된 배아 수_결측'] = df['해동된 배아 수'].apply(lambda x: 1 if x == -1 else 0)
    df['해동 난자 수_결측'] = df['해동 난자 수'].apply(lambda x: 1 if x == -1 else 0)
    df['수집된 신선 난자 수_결측'] = df['수집된 신선 난자 수'].apply(lambda x: 1 if x == -1 else 0)
    df['저장된 신선 난자 수_결측'] = df['저장된 신선 난자 수'].apply(lambda x: 1 if x == -1 else 0)
    df['혼합된 난자 수_결측'] = df['혼합된 난자 수'].apply(lambda x: 1 if x == -1 else 0)
    df['파트너 정자와 혼합된 난자 수_결측'] = df['파트너 정자와 혼합된 난자 수'].apply(lambda x: 1 if x == -1 else 0)
    df['기증자 정자와 혼합된 난자 수_결측'] = df['기증자 정자와 혼합된 난자 수'].apply(lambda x: 1 if x == -1 else 0)

    donor_age = {'알 수 없음': -1, '만20세 이하': 0, '만21-25세': 1, '만26-30세': 2,
                '만31-35세': 3, '만36-40세': 4, '만41-45세': 5}

    df['난자 기증자 나이'] = [donor_age[age] for age in df['난자 기증자 나이']]
    df['정자 기증자 나이'] = [donor_age[age] for age in df['정자 기증자 나이']]

    df['난자 기증자 나이_불명'] = df['난자 기증자 나이'].apply(lambda x: 1 if x == -1 else 0)
    df['정자 기증자 나이_불명'] = df['정자 기증자 나이'].apply(lambda x: 1 if x == -1 else 0)

    df['동결 배아 사용 여부'] = df['동결 배아 사용 여부'].fillna(-1)
    df['신선 배아 사용 여부'] = df['신선 배아 사용 여부'].fillna(-1)
    df['기증 배아 사용 여부'] = df['기증 배아 사용 여부'].fillna(-1)

    df['동결 배아 사용 여부_결측'] = df['동결 배아 사용 여부'].apply(lambda x: 1 if x == -1 else 0)
    df['신선 배아 사용 여부_결측'] = df['신선 배아 사용 여부'].apply(lambda x: 1 if x == -1 else 0)
    df['기증 배아 사용 여부_결측'] = df['기증 배아 사용 여부'].apply(lambda x: 1 if x == -1 else 0)

    df['대리모 여부'] = df['대리모 여부'].fillna(-1)
    df['대리모 여부_결측'] = df['대리모 여부'].apply(lambda x: 1 if x == -1 else 0)

    df['난자 해동 경과일'] = df['난자 해동 경과일'].fillna(-1)
    df['배아 해동 경과일'] = df['배아 해동 경과일'].fillna(-1)
    df['난자 혼합 경과일'] = df['난자 혼합 경과일'].fillna(-1)
    df['배아 이식 경과일'] = df['배아 이식 경과일'].fillna(-1)
    df['난자 채취 경과일'] = df['난자 채취 경과일'].fillna(-1)

    df['난자 해동 경과일_결측'] = df['난자 해동 경과일'].apply(lambda x: 1 if x == -1 else 0)
    df['배아 해동 경과일_결측'] = df['배아 해동 경과일'].apply(lambda x: 1 if x == -1 else 0)
    df['난자 혼합 경과일_결측'] = df['난자 혼합 경과일'].apply(lambda x: 1 if x == -1 else 0)
    df['배아 이식 경과일_결측'] = df['배아 이식 경과일'].apply(lambda x: 1 if x == -1 else 0)
    df['난자 채취 경과일_결측'] = df['난자 채취 경과일'].apply(lambda x: 1 if x == -1 else 0)

    # Category형 Feature 변환
    for feature in cat_features:
        if feature in df.columns:
            try:
                df[feature] = df[feature].astype('int')
            except:
                pass
            finally:
                df[feature] = df[feature].astype('category')

    df = df.drop(['특정 시술 유형', '배아 생성 주요 이유'], axis=1)

    return df

In [ ]:
train_clean = cleansing(train)
test_clean = cleansing(test)

In [ ]:
train_clean

In [ ]:
train_clean.info()

## Train

In [ ]:
train_clean.isna().sum().sum(), test_clean.isna().sum().sum()

In [ ]:
X = train_clean.drop(['임신 성공 여부'], axis=1)
y = train_clean['임신 성공 여부']

In [ ]:
X

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Stratified KFold 설정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Optuna objective 함수 정의
def objective(trial):
    # 하이퍼파라미터 탐색 범위 설정
    params = {
        "iterations": trial.suggest_int("iterations", 100, 500),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.03, 0.3),
        "depth": trial.suggest_int("depth", 4, 12),
        'cat_features': cat_features,  # 카테고리형 특성 리스트
        'random_state': 42,
        'eval_metric': 'AUC',
        'metric_period': 1,
        'task_type': 'GPU',  # GPU 사용
    }

    auc_scores = []

    # Stratified KFold 교차 검증
    for train_index, valid_index in skf.split(X, y):
        X_tr, X_val = X.iloc[train_index], X.iloc[valid_index]
        y_tr, y_val = y.iloc[train_index], y.iloc[valid_index]

        # 데이터 로드
        train_data = Pool(X_tr, label=y_tr, cat_features=params['cat_features'])
        valid_data = Pool(X_val, label=y_val, cat_features=params['cat_features'])

        # 모델 학습
        model = CatBoostClassifier(**params)
        model.fit(train_data, eval_set=valid_data, early_stopping_rounds=50, verbose=False)

        # 예측 및 ROC-AUC 계산
        y_pred_proba = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred_proba)
        auc_scores.append(auc)

    # 최종 평균 AUC 반환
    return np.mean(auc_scores)

In [ ]:
# Optuna 스터디 생성
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=30)  # 원하는 만큼의 트라이얼을 지정 (예: 50)

# 최적의 하이퍼파라미터 출력
print(f"Best hyperparameters: {study.best_params}")

## Predict

In [ ]:
# 최적의 모델로 전체 데이터 학습
# best_params = {'iterations': 459, 'learning_rate': 0.09376448430963388, 'depth': 6}
best_params = study.best_params
best_model = CatBoostClassifier(**best_params, cat_features=cat_features, random_state=42, task_type='GPU',
                                eval_metric='AUC', metric_period=1,
                                )


# 전체 데이터를 학습
train_pool = Pool(X_train, label=y_train, cat_features=cat_features)
best_model.fit(train_pool, verbose=5)

In [ ]:
auc_score = roc_auc_score(y_valid, best_model.predict_proba(X_valid)[:, 1])
logloss_score = log_loss(y_valid, best_model.predict_proba(X_valid)[:, 1])
print(f"AUC Score: {auc_score}")
print(f"Logloss Score: {logloss_score}")

In [ ]:
y_test_pred_proba = best_model.predict_proba(test_clean)[:, 1]

## Submission

In [ ]:
path = '/content/drive/MyDrive/data'

sample_submission = pd.read_csv(path+'/sample_submission.csv')
sample_submission['probability'] = y_test_pred_proba

In [ ]:
sample_submission.to_csv(path+'/submit_0225_2.csv', index=False)